In [1]:
# !pip install crewai crewai_tools langchain_community

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew, LLM

In [4]:
choice = 2
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
load_dotenv()

if choice==1:
    api_key = os.getenv("OPENAI_API_KEY")
    os.environ["OPENAI_API_KEY"] = api_key    
    llm = ChatOpenAI(model = "gpt-4o-mini",openai_api_key=api_key)

elif choice ==2:
    
    os.environ["OPENAI_API_KEY"] = "NA"
    llm = ChatOpenAI(
        model = "ollama/llama3.2",
        base_url = "http://localhost:11434"
    )
# else:
#     from langchain_community.llms import HuggingFaceHub
#     llm = HuggingFaceHub(
#         repo_id="HuggingFaceH4/zephyr-7b-beta",
#         huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_TOKEN"),
#         task="text-generation",
#     )

In [5]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
    llm = llm,
	verbose=True
)

In [6]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    llm = llm,
)

In [7]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, \
                         WebsiteSearchTool
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/concepts/crews"
)

In [8]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [9]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [10]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=1,
  memory=True
)

In [11]:
inputs = {
    "customer": "Resolve AI",
    "person": "Usman",
    "inquiry": "I need help with setting up a CrewAI for safe city "
               "how can I add memory to my crewAI? "
               "Can you provide guidance with steps?"
}
result = crew.kickoff(inputs=inputs)

2024-12-08 20:55:24,684 - 8492939328 - rag_storage.py-rag_storage:118 - ERROR: Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2024-12-08 20:55:25,715 - 8492939328 - rag_storage.py-rag_storage:118 - ERROR: Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


# Agent: Senior Support Representative
## Task: Resolve AI just reached out with a super important ask:
I need help with setting up a CrewAI for safe city how can I add memory to my crewAI? Can you provide guidance with steps?

Usman from Resolve AI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Using tool: Read website content
## Tool Input: 
"{\"section\": \"Setting up CrewAI for Safe City\", \"topic\": \"Adding Memory to CrewAI\"}"
## Tool Output: 
Crews - CrewAICrewAI home pageSearch CrewAI docscrewAIInc/crewAIcrewAIInc/crewAISearch...NavigationCore ConceptsCrewsGet StartedExamplesCrewAI home pageCommunityChangelogGet StartedIntroductionInstallationQuickstartCore ConceptsAgentsTasksCrewsFlowsKnowledgeLLMsProcessesCollaborationTrainingMemoryPlanningTestingCLIToolsUsing LangChain ToolsUsing LlamaInde

2024-12-08 20:56:00,134 - 8492939328 - rag_storage.py-rag_storage:89 - ERROR: Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers



2024-12-08 20:57:11,095 - 8492939328 - rag_storage.py-rag_storage:118 - ERROR: Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2024-12-08 20:57:12,226 - 8492939328 - rag_storage.py-rag_storage:118 - ERROR: Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


# Agent: Support Quality Assurance Specialist
## Task: Review the response drafted by the Senior Support Representative for Resolve AI's inquiry. Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.
Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.
Check for references and sources used to  find the information, ensuring the response is well-supported and leaves no questions unanswered.


# Agent: Support Quality Assurance Specialist
## Final Answer: 
The support representative's response should include the following information:

1. **Correct format for starting a crew's task execution**: The correct format for starting a crew's task execution is by using the kickoff() method.
2. **Different ways to kick off a crew**: CrewAI offers different ways to kick off a crew, including:
 * kickoff_for_each(): This method kicks off a crew by executing each task se

2024-12-08 20:57:23,004 - 8492939328 - rag_storage.py-rag_storage:89 - ERROR: Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers



In [12]:
from IPython.display import display, Markdown
Markdown(result.raw)

The support representative's response should include the following information:

1. **Correct format for starting a crew's task execution**: The correct format for starting a crew's task execution is by using the kickoff() method.
2. **Different ways to kick off a crew**: CrewAI offers different ways to kick off a crew, including:
 * kickoff_for_each(): This method kicks off a crew by executing each task sequentially.
 * kickoff_async(): This method kicks off a crew asynchronously, which allows for more flexibility in terms of task execution.
 * kickoff_for_each_async(): This method kicks off a crew asynchronously and executes each task sequentially. Note that this is similar to the kickoff_for_each() method but uses asynchronous execution.

3. **Replay feature**: The replay feature in CrewAI allows users to replay from a specific task using the command-line interface (CLI). This can be useful for debugging or testing purposes.

4. **References and sources**: To verify the accuracy of this information, one can refer to the CrewAI documentation or contact their support team directly. Additionally, reputable sources such as GitHub or Stack Overflow may also provide valuable insights and examples.

5. **Helpful and friendly tone**: The response should be written in a clear and concise manner, avoiding technical jargon whenever possible. It's essential to use a helpful and friendly tone to ensure that the customer feels supported and confident in using CrewAI.

6. **No assumptions**: The support representative should avoid making assumptions about the customer's experience or expectations. Instead, they should focus on providing accurate information and addressing any questions or concerns raised by the customer.

7. **Full completion of customer's inquiry**: The response should fully address the customer's inquiry, incorporating all relevant feedback and improvements. This ensures that the customer feels supported and has a clear understanding of how to use CrewAI.

By following these guidelines, the support representative can provide a comprehensive and accurate response that meets the high-quality standards expected for customer support.